In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
from skforecast.datasets import fetch_dataset

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.size'] = 10

# Modelado y Forecasting
# ==============================================================================
import sklearn
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
import shap


# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('once')

color = '\033[1m\033[38;5;208m' 
print(f"{color}Versión skforecast: {skforecast.__version__}")
print(f"{color}Versión scikit-learn: {sklearn.__version__}")
print(f"{color}Versión pandas: {pd.__version__}")
print(f"{color}Versión numpy: {np.__version__}")

In [ ]:
Df_O_salidas = pd.read_excel('./Data resumida.xlsx')
Df_O_salidas['FECHA_RIGE']= pd.to_datetime(Df_O_salidas['FECHA_RIGE'],format='%d-%m-%Y')
fecha_limite = pd.to_datetime('30-06-2024', format='%d-%m-%Y')
Df_O_salidas = Df_O_salidas[Df_O_salidas['FECHA_RIGE']<=fecha_limite]
Df_O_salidas['Year_month'] = Df_O_salidas['FECHA_RIGE']+ pd.offsets.MonthEnd(0)
Df_O_salidas = Df_O_salidas.sort_values('Year_month')
Df_agrupado = Df_O_salidas.groupby(by=['Year_month', 'DEPTO_ACTUAL'])['Recuento_de_salida'].sum().reset_index()
valores_unicos = pd.unique(Df_O_salidas['DEPTO_ACTUAL'])
decer = Df_agrupado
  
df_unificado = []

for rest in valores_unicos :
    dfderecurrencia = Df_agrupado[Df_agrupado['DEPTO_ACTUAL'] == rest].copy()
    dfderecurrencia.set_index('Year_month', inplace=True)
    dfderecurrencia = dfderecurrencia.asfreq('M')
    dfderecurrencia.sort_index()
    # Añadir el departamento al DataFrame resultante
    dfderecurrencia['DEPTO_ACTUAL'] = rest
    dfderecurrencia['Recuento_de_salida'] = dfderecurrencia['Recuento_de_salida'].fillna(0)
    # Hacer un 'push' (append) de dfderecurrencia a df_unificado
    df_unificado.append(dfderecurrencia)
# Resetear el índice de df_unificado si es necesario

df_unificadopd = pd.concat(df_unificado, ignore_index=False)
#df_unificadopd.to_excel('./resultado.xlsx')
#Restaurante_unicos = pd.unique(Df_agrupado['DEPTO_ACTUAL'])
print(df_unificadopd)
#Df_agrupado['Year_month'] = pd.to_datetime(Df_agrupado['Year_month'])
#Df_agrupado = Df_agrupado.set_index(['Year_month','DEPTO_ACTUAL'])


# Mostrar los primeros resultados
#Df_agrupado.head()

In [ ]:
df_unificadopd2 = df_unificadopd.groupby(by=df_unificadopd.index)['Recuento_de_salida'].sum().reset_index()
df_unificadopd2.set_index('Year_month', inplace=True)
df_unificadopd2.index = pd.to_datetime(df_unificadopd2.index)
print(df_unificadopd2)

plt.figure(figsize=(10,10))
plt.plot(df_unificadopd2.index,df_unificadopd2['Recuento_de_salida'],scalex='M',marker='o')
# Formato del eje X
plt.xlabel('Fecha')  # Etiqueta del eje X
plt.ylabel('Valores')  # Etiqueta del eje Y
plt.title('Gráfico de Líneas')
plt.xticks(rotation=45) 
plt.show()

La Gafica linea muestra que hay una tendencia a la alza con una etacionalidad aditiva sin contar el periodo 2020 que esa anormalidad se abduce que es por la pandemia

In [ ]:
steps = 18
data_train = df_unificadopd2[:-steps]
data_test  = df_unificadopd2[-steps:]

print(
    f"Train dates : {data_train.index.min()} --- "
    f"{data_train.index.max()}  (n={len(data_train)})"
)
print(
    f"Test dates  : {data_test.index.min()} --- "
    f"{data_test.index.max()}  (n={len(data_test)})"
)
fig, ax = plt.subplots(figsize=(8, 6))
data_train['Recuento_de_salida'].plot(ax=ax, label='train')
data_test['Recuento_de_salida'].plot(ax=ax, label='test')
ax.legend()

In [ ]:
# Create and train forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 6
             )
forecaster.fit(y=data_train['Recuento_de_salida'])
forecaster 

In [ ]:
# Supongamos que `data_train` y `data_test` ya están definidos, y contienen un índice de fechas.

# Generar las predicciones
steps = 18
predictions = pd.Series(
    data=forecaster.predict(steps=steps),
    name='Predicción'
)

print(predictions)
# Crear un índice de fechas para las predicciones
pred_index = pd.date_range(
    start=data_train.index[-1] + pd.Timedelta(days=1), 
    periods=steps, 
    freq='M'  # Ajusta la frecuencia según tu serie temporal
)

# Alinear las predicciones con el índice
predictions.index = pred_index

# Crear un DataFrame con las predicciones (ya alineado)
predictions_df = pd.DataFrame(predictions)


print(predictions_df)
# Graficar los datos
fig, ax = plt.subplots(figsize=(10, 5))
data_train['Recuento_de_salida'].plot(ax=ax, label='train')
data_test['Recuento_de_salida'].plot(ax=ax, label='test')
predictions_df['Predicción'].plot(ax=ax, label='predictions', color='black')
ax.legend()
plt.show()

print(data_train.index.max(),predictions_df.index.min())


In [ ]:
error_mse = mean_squared_error(
                y_true = data_test['Recuento_de_salida'],
                y_pred = predictions
            )
print(f"Test error (MSE): {error_mse}")


steps2= 18

forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 12 # This value will be replaced in the grid search
             )

lags_grid = [6, 12]
# Definir el espacio de búsqueda para los hiperparámetros
param_grid = {
    'n_estimators': [100, 250],  # Número de árboles en el bosque
    'max_depth': [3, 8],          # Profundidad máxima de los árboles
}




# Realizar la búsqueda de hiperparámetros
results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = data_train['Recuento_de_salida'],
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = steps,
                   metric             = 'mean_squared_error',
                   initial_train_size = int(len(data_train)*0.5),
                   fixed_train_size   = False,
                   refit              = False,
                   return_best        = True,
                   n_jobs             = 'auto',
                   verbose            = False
               )



dedegg
